In [1]:
import torch
import torch.nn as nn

In [2]:
class Residual(nn.Module):
    def __init__(self, fn: nn.Module):
        super().__init__()
        self.fn = fn

    def forward(self, x):
        return x + self.fn(x)

In [3]:
class ResidualModule(nn.Module):
    def __init__(self, input_dim, hidden_dim, norm=nn.BatchNorm1d, drop_prob=0.1):
        super().__init__()
        self.modules = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            norm(hidden_dim),
            nn.ReLU(),
            nn.Dropout(drop_prob),
            nn.Linear(hidden_dim, input_dim),
            norm(input_dim),
        )

    def forward(self):
        return nn.Sequential(
            Residual(self.modules),
            nn.ReLU()
        )

In [3]:
def ResNet(input_dim, hidden_dim=100, num_blocks=3, output_dim=2, norm=nn.BatchNorm1d, drop_prob=0.1):
    modules = [
        nn.Linear(input_dim, hidden_dim),
        nn.ReLU()
    ]
    for i in range(num_blocks):
        modules.append(ResidualModule(hidden_dim, hidden_dim // 2, norm, drop_prob))
    modules.append(nn.Linear(hidden_dim, output_dim))
    return nn.Sequential(*modules)

In [17]:
import os
import numpy as np
from PIL import Image
from tqdm import tqdm

X = []
y = []
manipulated_path = './data/manipulated'
original_path = './data/original'
print("processing fake datasets")
for img_name in tqdm(os.listdir(manipulated_path)):
    X.append(np.array(Image.open(f'{manipulated_path}/{img_name}')))
    y.append(0)
print("processing true datasets")
for img_name in tqdm(os.listdir(original_path)):
    X.append(np.array(Image.open(f'{original_path}/{img_name}')))
    y.append(1)
X = np.array(X)
y = np.array(y).reshape((-1, 1))
print(X.shape)
print(y.shape)

processing fake datasets


100%|██████████| 8000/8000 [00:26<00:00, 305.71it/s]


processing true datasets


100%|██████████| 4000/4000 [00:16<00:00, 244.05it/s]


(12000, 299, 299, 3)
(12000, 1)
